In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_table("../../data/raw/TB_BAJAS_CL.tsv", encoding="latin-1")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99025 entries, 0 to 99024
Data columns (total 3 columns):
POLICY_NUMBER          99025 non-null int64
CANCELLATION_DATE      99025 non-null object
CANCELLATION_REASON    16917 non-null object
dtypes: int64(1), object(2)
memory usage: 2.3+ MB


In [3]:
df.head()

,POLICY_NUMBER,CANCELLATION_DATE,CANCELLATION_REASON
0,1436751829999,31/08/2015,NaN
1,1378134259999,28/02/2015,NaN
2,278935929999,29/02/2004,NaN
3,1256190669999,31/03/2014,NaN
4,1345439839999,30/11/2014,NaN


In [4]:
df_poliza = pd.read_table("../../data/raw/TB_POLIZAS_CL.tsv", encoding="latin-1")
df_poliza.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4070858 entries, 0 to 4070857
Data columns (total 20 columns):
PROCESS_DATE            object
CUSTOMER_ID             int64
POLICY_ORIGINAL         float64
POLICY_NUMBER           int64
DATE_ISSUE              object
DATE_COMMENCENT_RISK    object
SUM_ASSURED             float64
PREMIUM                 object
PAYMENT_FREQUENCY       object
COD_VEH_MAKE            int64
VEH_MAKE                object
VEH_MODEL               object
VEH_YEAR                int64
CHANEL_ID               int64
CHANNEL                 object
NAME_OF_PRODUCT         object
TYPE_PRODUCT            object
COVERAGE_DETAILS        object
AGENT_ID                float64
SUB_AGENT_ID            float64
dtypes: float64(4), int64(5), object(11)
memory usage: 621.2+ MB


In [5]:
df_can_pol = df.merge(df_poliza[["POLICY_NUMBER", "CUSTOMER_ID"]], how="inner", left_on=["POLICY_NUMBER"], 
                             right_on=["POLICY_NUMBER"], suffixes=['', '_poliza'])
df_can_pol.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116564 entries, 0 to 116563
Data columns (total 4 columns):
POLICY_NUMBER          116564 non-null int64
CANCELLATION_DATE      116564 non-null object
CANCELLATION_REASON    20056 non-null object
CUSTOMER_ID            116564 non-null int64
dtypes: int64(2), object(2)
memory usage: 4.4+ MB


In [6]:
bd = df_can_pol.drop_duplicates()
bd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98796 entries, 0 to 116563
Data columns (total 4 columns):
POLICY_NUMBER          98796 non-null int64
CANCELLATION_DATE      98796 non-null object
CANCELLATION_REASON    16917 non-null object
CUSTOMER_ID            98796 non-null int64
dtypes: int64(2), object(2)
memory usage: 3.8+ MB


In [7]:
#convierto el due_date en fecha, para luego obtener el siguiente mes (correspondiente al servicio)
from datetime import datetime, timedelta
bd["fecha"] = bd.apply(lambda x : datetime.strptime(x.CANCELLATION_DATE, "%d/%m/%Y"), axis=1)
bd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98796 entries, 0 to 116563
Data columns (total 5 columns):
POLICY_NUMBER          98796 non-null int64
CANCELLATION_DATE      98796 non-null object
CANCELLATION_REASON    16917 non-null object
CUSTOMER_ID            98796 non-null int64
fecha                  98796 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 4.5+ MB


/home/seba/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
bd.head()

,POLICY_NUMBER,CANCELLATION_DATE,CANCELLATION_REASON,CUSTOMER_ID,fecha
0,1436751829999,31/08/2015,NaN,6131994,2015-08-31
1,1378134259999,28/02/2015,NaN,3144725,2015-02-28
2,1256190669999,31/03/2014,NaN,11752620,2014-03-31
3,1345439839999,30/11/2014,NaN,3165817,2014-11-30
4,711055139999,31/05/2009,NaN,13146367,2009-05-31


In [9]:
bd["mes"] = bd.fecha.apply(lambda x : x if pd.isnull(x) else int(x.month) )
bd["anio"] = bd.fecha.apply(lambda x : x if pd.isnull(x) else int(x.year) )
bd["corresp_date"] = bd.fecha.apply(lambda x : x if pd.isnull(x) else "{}-{:02}".format(x.year, x.month) )
bd.info()

/home/seba/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/seba/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


<class 'pandas.core.frame.DataFrame'>
Int64Index: 98796 entries, 0 to 116563
Data columns (total 8 columns):
POLICY_NUMBER          98796 non-null int64
CANCELLATION_DATE      98796 non-null object
CANCELLATION_REASON    16917 non-null object
CUSTOMER_ID            98796 non-null int64
fecha                  98796 non-null datetime64[ns]
mes                    98796 non-null int64
anio                   98796 non-null int64
corresp_date           98796 non-null object
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 6.8+ MB


/home/seba/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
df_poliza[df_poliza["POLICY_NUMBER"] == 1436751829999]

,PROCESS_DATE,CUSTOMER_ID,POLICY_ORIGINAL,POLICY_NUMBER,DATE_ISSUE,DATE_COMMENCENT_RISK,SUM_ASSURED,PREMIUM,PAYMENT_FREQUENCY,COD_VEH_MAKE,VEH_MAKE,VEH_MODEL,VEH_YEAR,CHANEL_ID,CHANNEL,NAME_OF_PRODUCT,TYPE_PRODUCT,COVERAGE_DETAILS,AGENT_ID,SUB_AGENT_ID
3455191,31/07/2015,6131994,8.070117e+12,1436751829999,27/06/2015,19/09/2000,27300.0,"437,20",MENSUAL,6614,FORD,SIERRA,1989,0,Aca,AUTO,"RC, ROBO E INCENDIO PARCIAL Y TOTAL Y DESTRUCC...",DISCONTINUADO,NaN,NaN


In [11]:
bd.sort_values(["anio", "mes"], inplace=True)
bd.head()

/home/seba/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,POLICY_NUMBER,CANCELLATION_DATE,CANCELLATION_REASON,CUSTOMER_ID,fecha,mes,anio,corresp_date
122,279811759999,31/03/2004,NaN,1387814,2004-03-31,3,2004,2004-03
717,257502869999,31/03/2004,NaN,1782094,2004-03-31,3,2004,2004-03
921,263038699999,31/03/2004,NaN,1429159,2004-03-31,3,2004,2004-03
1338,266126589999,31/03/2004,NaN,8707059,2004-03-31,3,2004,2004-03
1378,282654169999,31/03/2004,NaN,10741277,2004-03-31,3,2004,2004-03


In [14]:
#ver las razones de cancelación
bd.CANCELLATION_REASON.value_counts().index[3]

'VV - Vta. Vehículo - Ya Aseguro en LA CAJA'

In [18]:
columns = bd.corresp_date.unique()
row = bd.CUSTOMER_ID.unique()

# creando matriz cancelación mes x cliente

In [15]:
#creo el dataset vacio
df_m = pd.DataFrame(columns=columns)
df_m.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Columns: 166 entries, 2004-03 to 2017-12
dtypes: object(166)
memory usage: 0.0+ bytes


In [16]:
#agrupo por cliente y mes_anio y cuento cuantas polizas distintas tiene
groups = bd.groupby(["CUSTOMER_ID", "corresp_date"])["POLICY_NUMBER"].count()

In [35]:
groups

CUSTOMER_ID  corresp_date
1690         2005-09         1
             2006-03         1
             2007-06         1
             2009-03         1
             2010-01         1
             2015-01         1
             2015-07         1
5575         2012-01         1
             2012-10         1
             2015-08         1
             2016-01         1
             2017-09         2
7968         2011-12         1
9028         2013-02         1
             2016-09         1
             2016-10         1
9860         2009-09         1
             2017-09         1
11820        2016-05         1
14923        2007-03         1
             2010-07         1
             2011-01         1
             2011-04         1
             2012-06         1
             2014-01         1
             2014-06         1
             2014-07         1
             2014-11         1
             2014-12         1
             2015-03         1
                            ..
21079290     

In [17]:
for id_cliente in groups.index.levels[0]:
    df_m.loc[id_cliente] = groups[id_cliente].T

In [18]:
df_m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41754 entries, 1690 to 21260067
Columns: 166 entries, 2004-03 to 2017-12
dtypes: float64(166)
memory usage: 53.2 MB


In [19]:
df_m.head()

,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,2004-12,...,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
7968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [43]:
df_m.to_csv("../../data/processed/BD_ID_CLIENTE_N.CANCELACIONES_X_MES.csv")

# Creando matriz cancelación segun motivo

retomo de arriba (antes de la cabecera previa)

In [15]:
#Si la cancelación, no es negativa (es decir, va a volver a la empresa) => 0, sino 1
bd["MotivoN"] = 1
bd.loc[bd.CANCELLATION_REASON == "VV - Vta. Vehículo - Ya Aseguro en LA CAJA", "MotivoN"] = 0

/home/seba/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/seba/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [16]:
bd.head()

,POLICY_NUMBER,CANCELLATION_DATE,CANCELLATION_REASON,CUSTOMER_ID,fecha,mes,anio,corresp_date,MotivoN
122,279811759999,31/03/2004,NaN,1387814,2004-03-31,3,2004,2004-03,1
717,257502869999,31/03/2004,NaN,1782094,2004-03-31,3,2004,2004-03,1
921,263038699999,31/03/2004,NaN,1429159,2004-03-31,3,2004,2004-03,1
1338,266126589999,31/03/2004,NaN,8707059,2004-03-31,3,2004,2004-03,1
1378,282654169999,31/03/2004,NaN,10741277,2004-03-31,3,2004,2004-03,1


In [19]:
#creo el dataset vacio
df_m = pd.DataFrame(columns=columns)
df_m.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Columns: 166 entries, 2004-03 to 2017-12
dtypes: object(166)
memory usage: 0.0+ bytes


In [20]:
#agrupo por cliente y mes_anio y cuento cuantas polizas distintas tiene
groups = bd.groupby(["CUSTOMER_ID", "corresp_date"])["MotivoN"].sum()

In [21]:
for id_cliente in groups.index.levels[0]:
    df_m.loc[id_cliente] = groups[id_cliente].T

In [22]:
df_m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41754 entries, 1690 to 21260067
Columns: 166 entries, 2004-03 to 2017-12
dtypes: float64(166)
memory usage: 53.2 MB


In [23]:
df_m.head()

,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,2004-12,...,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
7968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [24]:
df_m.to_csv("../../data/processed/BD_ID_CLIENTE_N.CANCELACIONES.negativas_X_MES.csv")